In [ ]:
!pip install -q openai

In [ ]:
import time
import psutil
import openai
import pandas as pd
from google.colab import drive

# mount google drive
drive.mount('/content/drive')

# Set your OpenAI API key
openai.api_key = ""

Mounted at /content/drive


In [ ]:
messages = [
    {"role": "system", "content": "Assistant is an intelligent chatbot. First read the twitter post, then identify the travel mode and the person's sentiment. Identify the reason only if the sentiment is positive or negative. Leave the reason blank if the sentiment is neutral."},
    {"role": "system", "content": "Write in the following format: Travel Mode: xxx, Sentiment: xxx, Reason: xxx. Don't write in any other format."},
    {"role": "user", "content": "I've been waiting a long time for the subway. I have been waiting for half an hour already there is no subway coming."},
    {"role": "assistant", "content": "Travel Mode: subway\n"
                      "Sentiment: negative\n"
                      "Reason: long waiting time"},
    {"role": "user", "content": "I like eating noodles"},
    {"role": "assistant", "content": "not related content"},
    {"role": "user", "content": "I want to give credit to the bus service in New York. There was flood last week but the bus service was still running. I was able to make it to work on time."},
    {"role": "assistant", "content": "Travel Mode: bus\n"
                      "Sentiment: positive\n"
                      "Reason: the bus servies was still running even there was flood last week"},
    {"role": "user", "content": "I was so disappointed. I got a flight to Seattle last week. It was overbooked and I was bumped from the airplane with little compensation."},
    {"role": "assistant", "content": "Travel Mode: airplane\n"
                      "Sentiment: negative\n"
                      "Reason: bumped from the airplane and received little compensation"},
    {"role": "user", "content": "I took taxi to the office. The weather was sunny."},
    {"role": "assistant", "content": "Travel Mode: taxi\n"
                      "Sentiment: neutral"}
]

In [ ]:
# load file
file_path = '/content/drive/MyDrive/LLM Research /Data/processed_1000_1999.csv'
data = pd.read_csv(file_path)

In [ ]:
data

,Unnamed: 0,GLOBAL_ID,LOCAL_ID,processed_txt,date,_date_,WoY
0,1000,1000,1969,second system subway and legalized mushrooms,2021-08-10 15:53:43+00:00,2021-08-10,2021-32
1,1001,1001,1971,i am at mta subway bay ridge ave (r) in new yo...,2021-08-10 15:37:24+00:00,2021-08-10,2021-32
2,1002,1002,1975,i am at mta subway bay ridge ave (r) in new yo...,2021-08-10 15:02:43+00:00,2021-08-10,2021-32
3,1003,1003,1979,texas lawmakers are derived of all the trash a...,2021-08-10 14:30:09+00:00,2021-08-10,2021-32
4,1004,1004,1980,i am at mta subway hoyt and schermerhorn sts (...,2021-08-10 14:28:16+00:00,2021-08-10,2021-32
...,...,...,...,...,...,...,...
995,1995,1995,3999,you are not wrong. every morning i wake up and...,2020-07-23 14:07:40+00:00,2020-07-23,2020-29
996,1996,1996,4001,brother one year i was saucy in coney island i...,2020-07-23 14:06:37+00:00,2020-07-23,2020-29
997,1997,1997,4004,this older white man is really on the path wit...,2020-07-23 13:26:52+00:00,2020-07-23,2020-29
998,1998,1998,4007,you do realize people tune in to watch train w...,2020-07-23 13:01:56+00:00,2020-07-23,2020-29


In [ ]:
reply = ''
reply_list = []

for index, row in data.iterrows():
    messages.append({
        "role": "user", "content": row['processed_txt']
    })
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.2
    )
    reply = response.choices[0].message.content
    reply_list.append(reply)
    messages.pop()

print(reply_list)

['not related content', 'Travel Mode: subway\nSentiment: neutral', 'not related content', 'not related content', 'Travel Mode: subway\nSentiment: neutral', 'Travel Mode: subway\nSentiment: neutral', 'Travel Mode: subway\nSentiment: neutral', 'not related content', 'Travel Mode: cab\nSentiment: negative\nReason: forgot keys, train suspension', 'Travel Mode: subway\nSentiment: neutral', 'Travel Mode: subway\nSentiment: neutral', 'Travel Mode: subway\nSentiment: negative\nReason: subway outage between 96th and 145th stations', 'Travel Mode: subway\nSentiment: negative\nReason: the B train is running slowly', 'Travel Mode: subway\nSentiment: negative\nReason: waiting for another train to pass at President Street', 'Travel Mode: subway\nSentiment: neutral', 'not related content', 'Travel Mode: train\nSentiment: negative\nReason: the train is too hot', 'Travel Mode: train\nSentiment: positive\nReason: highlight of the summer', 'Travel Mode: subway\nSentiment: negative\nReason: no 3 train run

In [ ]:
reply_list

['not related content',
 'Travel Mode: subway\nSentiment: neutral',
 'not related content',
 'not related content',
 'Travel Mode: subway\nSentiment: neutral',
 'Travel Mode: subway\nSentiment: neutral',
 'Travel Mode: subway\nSentiment: neutral',
 'not related content',
 'Travel Mode: cab\nSentiment: negative\nReason: forgot keys, train suspension',
 'Travel Mode: subway\nSentiment: neutral',
 'Travel Mode: subway\nSentiment: neutral',
 'Travel Mode: subway\nSentiment: negative\nReason: subway outage between 96th and 145th stations',
 'Travel Mode: subway\nSentiment: negative\nReason: the B train is running slowly',
 'Travel Mode: subway\nSentiment: negative\nReason: waiting for another train to pass at President Street',
 'Travel Mode: subway\nSentiment: neutral',
 'not related content',
 'Travel Mode: train\nSentiment: negative\nReason: the train is too hot',
 'Travel Mode: train\nSentiment: positive\nReason: highlight of the summer',
 'Travel Mode: subway\nSentiment: negative\nReas

In [ ]:
# Initialize lists to store extracted data
travel_mode = []
sentiment = []
reason = []

# Loop through each item in the reply_list
for item in reply_list:
    # Initialize variables to store extracted values
    mode = ''
    sent = ''
    rsn = ''

    # Check if "Travel Mode" exists
    if "Travel Mode" in item:
        # Split the string by 'Sentiment:' to get the mode and sentiment
        parts = item.split('Sentiment:')

        if len(parts) > 1:
            # Extract the mode
            mode = parts[0].replace('Travel Mode:', '').strip()

            # Extract the sentiment
            sent = parts[1].strip()

            # Check if "Reason" exists and extract it
            reason_parts = sent.split('Reason:')
            if len(reason_parts) > 1:
                sent = reason_parts[0].strip()
                rsn = reason_parts[1].strip()

    # If "Travel Mode" is not found, store the entire item in the "Travel Mode" column
    if not mode:
        mode = item

    # Append the extracted values to their respective lists
    travel_mode.append(mode)
    sentiment.append(sent)
    reason.append(rsn)

# Create a DataFrame from the extracted data
df = pd.DataFrame({
    'Travel Mode': travel_mode,
    'Sentiment': sentiment,
    'Reason': reason
})

In [ ]:
reply_list

['not related content',
 'Travel Mode: subway\nSentiment: Neutral',
 'Travel Mode: subway\nSentiment: Neutral',
 'not related content',
 'Travel Mode: subway\nSentiment: Neutral',
 'Travel Mode: subway\nSentiment: Neutral',
 'Travel Mode: subway\nSentiment: negative\nReason: masks are required on the subway again',
 'not related content',
 'Travel Mode: cab\nSentiment: negative\nReason: forgot keys and had to take a cab due to suspended train']

In [ ]:
df

,Travel Mode,Sentiment,Reason
0,not related content,,
1,subway,neutral,
2,not related content,,
3,not related content,,
4,subway,neutral,
...,...,...,...
995,subway,positive,the virus is under control in NYC
996,F train,negative,got home very late and was drunk
997,not related content,,
998,not related content,,


In [ ]:
# merge it back to the original dataframe
merged_df = pd.concat([data, df], axis=1)

In [ ]:
merged_df['processed_txt'][8]

"i forgot my keys this morning (roommates are home thank fuck) and had to get a cab to work because the only train that runs to morning baby's neighborhood was suspended, but how is your day?"

In [ ]:
# human_verification = ['incorrect', 'correct', 'correct', 'incorrect', 'correct', 'correct', 'correct', 'incorrect', 'incorrect travel mode, correct sentiment']
# merged_df['Human Verification'] = human_verification

In [ ]:
merged_df

,Unnamed: 0,GLOBAL_ID,LOCAL_ID,processed_txt,date,_date_,WoY,Travel Mode,Sentiment,Reason
0,1000,1000,1969,second system subway and legalized mushrooms,2021-08-10 15:53:43+00:00,2021-08-10,2021-32,not related content,,
1,1001,1001,1971,i am at mta subway bay ridge ave (r) in new yo...,2021-08-10 15:37:24+00:00,2021-08-10,2021-32,subway,neutral,
2,1002,1002,1975,i am at mta subway bay ridge ave (r) in new yo...,2021-08-10 15:02:43+00:00,2021-08-10,2021-32,not related content,,
3,1003,1003,1979,texas lawmakers are derived of all the trash a...,2021-08-10 14:30:09+00:00,2021-08-10,2021-32,not related content,,
4,1004,1004,1980,i am at mta subway hoyt and schermerhorn sts (...,2021-08-10 14:28:16+00:00,2021-08-10,2021-32,subway,neutral,
...,...,...,...,...,...,...,...,...,...,...
995,1995,1995,3999,you are not wrong. every morning i wake up and...,2020-07-23 14:07:40+00:00,2020-07-23,2020-29,subway,positive,the virus is under control in NYC
996,1996,1996,4001,brother one year i was saucy in coney island i...,2020-07-23 14:06:37+00:00,2020-07-23,2020-29,F train,negative,got home very late and was drunk
997,1997,1997,4004,this older white man is really on the path wit...,2020-07-23 13:26:52+00:00,2020-07-23,2020-29,not related content,,
998,1998,1998,4007,you do realize people tune in to watch train w...,2020-07-23 13:01:56+00:00,2020-07-23,2020-29,not related content,,


In [ ]:
reply = ''
reply_list = []

verify_message = [{
    "role": "system", "content": "You are a helpful verifier. Look at the twitter post and the travel mode & sentiment identified, tell me whether the travel mode and sentiment identified from the post is correct or incorrect. Only reply 'correct' or 'incorrect'.",
    # "role": "user", "content": "Twitter Post: second system subway and legalized mushrooms, Travel Mode: not related content, Sentiment:",
    # "role": "assistant", "content": "correct",
    # "role": "user", "content": "Twitter Post: i am at mta subway bay ridge ave (r) in new york, ny https: and and t.co and m319xptxjm, Travel Mode: subway, Sentiment: Neutral",
    # "role": "assistant", "content": "correct",
}]
for index, row in merged_df.iterrows():
  verify_message.append({"role": "user", "content": f"Twitter Post: {row['processed_txt']}, Travel Mode: {row['Travel Mode']}, Sentiment: {row['Sentiment']}"})
  print(verify_message)
  response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=verify_message,
      temperature=0.2
  )
  reply = response['choices'][0]['message']['content']
  print('reply', reply)
  reply_list.append(reply)
  verify_message.pop()

[{'role': 'system', 'content': "You are a helpful verifier. Look at the twitter post and the travel mode & sentiment identified, tell me whether the travel mode and sentiment identified from the post is correct or incorrect. Only reply 'correct' or 'incorrect'."}, {'role': 'user', 'content': 'Twitter Post: second system subway and legalized mushrooms, Travel Mode: not related content, Sentiment: '}]
reply Incorrect
[{'role': 'system', 'content': "You are a helpful verifier. Look at the twitter post and the travel mode & sentiment identified, tell me whether the travel mode and sentiment identified from the post is correct or incorrect. Only reply 'correct' or 'incorrect'."}, {'role': 'user', 'content': 'Twitter Post: i am at mta subway bay ridge ave (r) in new york, ny https: and and t.co and m319xptxjm, Travel Mode: subway, Sentiment: Neutral'}]
reply correct
[{'role': 'system', 'content': "You are a helpful verifier. Look at the twitter post and the travel mode & sentiment identified

In [ ]:
merged_df['Self-Verification'] = reply_list

In [ ]:
merged_df

,GLOBAL_ID,LOCAL_ID,processed_txt,date,_date_,WoY,Travel Mode,Sentiment,Reason,Human Verification,Self-Verification
0,1000,1969,second system subway and legalized mushrooms,2021-08-10 15:53:43+00:00,2021-08-10,2021-32,not related content,,,incorrect,Incorrect
1,1001,1971,i am at mta subway bay ridge ave (r) in new yo...,2021-08-10 15:37:24+00:00,2021-08-10,2021-32,subway,Neutral,,correct,correct
2,1002,1975,i am at mta subway bay ridge ave (r) in new yo...,2021-08-10 15:02:43+00:00,2021-08-10,2021-32,subway,Neutral,,correct,correct
3,1003,1979,texas lawmakers are derived of all the trash a...,2021-08-10 14:30:09+00:00,2021-08-10,2021-32,not related content,,,incorrect,Incorrect
4,1004,1980,i am at mta subway hoyt and schermerhorn sts (...,2021-08-10 14:28:16+00:00,2021-08-10,2021-32,subway,Neutral,,correct,correct
5,1005,1982,"i am at mta subway court st (r) in brooklyn, n...",2021-08-10 14:17:02+00:00,2021-08-10,2021-32,subway,Neutral,,correct,correct
6,1006,1983,masks on the subway again?,2021-08-10 14:16:19+00:00,2021-08-10,2021-32,subway,negative,masks are required on the subway again,correct,correct
7,1007,1984,all you had to do was follow the god damn trai...,2021-08-10 13:56:37+00:00,2021-08-10,2021-32,not related content,,,incorrect,Incorrect
8,1008,1985,i forgot my keys this morning (roommates are h...,2021-08-10 13:54:06+00:00,2021-08-10,2021-32,cab,negative,forgot keys and had to take a cab due to suspe...,"incorrect travel mode, correct sentiment",Incorrect


In [ ]:
# download the file

from google.colab import files
merged_df.to_csv('1000-1999 complete.csv')
files.download('1000-1999 complete.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>